In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 11:49:08 2019
@author: wangxinyi
"""
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import math

import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


conn= pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='root123',charset='utf8',db='car_evaluation')

all_data_df=pd.read_sql_query('''
select id,ty,concat(substr(t,1,9),'0:00') as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  , concat(max(x),'-',max(y))  as xy
from tianchi_01_train  group by id,ty,concat(substr(t,1,9),'0:00') ;
''',conn)
#where id<100

In [4]:
all_data_df.head(1)

,id,x,y,v,d,t,ty,xy
0,0,6.152038e+06,5.124873e+06,2.59,102,1110 11:58:19,拖网,6152038.42515299-5124873.3827915


In [2]:
from scipy.stats import entropy
from dateutil import parser

def t_p(t):
    #return t.size
    return (parser.parse(max(t))-parser.parse(min(t))).total_seconds()/t.size

def entropy_wxy(x):
    index_x={}
    for c in x:
        if c in index_x:
            index_x[c]=index_x[c]+1
        else:
            index_x[c]=1
    vv=np.array(list(index_x.values()))    
    return entropy(vv, base=2)
def sub_num(x):
    return np.max(x)-np.min(x)

def distinct_count(xy):
    return len(set(xy))

def model_num(x):
    return np.mean(x.mode())

def kmeans_xy(xy):
    return xy.size



all_df=all_data_df.groupby(['id', 'ty']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num],'y': [np.mean, np.max,np.min,np.median,model_num,sub_num],
                                     'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count] ,'d':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count],
                                       't':t_p,
                                        'xy':distinct_count
                                      })

In [8]:
all_df.head(2)

x                                                          \
               mean          amax          amin        median     model_num   
id ty                                                                         
0  拖网  6.119351e+06  6.152038e+06  6.118352e+06  6.118352e+06  6.118352e+06   
1  拖网  6.091460e+06  6.102450e+06  6.049472e+06  6.102450e+06  6.102450e+06   

                                y                                            \
            sub_num          mean          amax          amin        median   
id ty                                                                         
0  拖网  33686.667453  5.130494e+06  5.130781e+06  5.124873e+06  5.130672e+06   
1  拖网  52978.013345  5.094050e+06  5.112874e+06  5.042857e+06  5.112760e+06   

           ...               v                         d                    \
           ...       model_num distinct_count       mean        std median   
id ty      ...                                                               
0  拖网      ...             0.0           21.0   4.613527  21.247770    0.0   
1  拖网      ...             0.0           57.0  56.153247  91.449382    0.0   

                                                        t             xy  
      entropy_wxy model_num distinct_count            t_p distinct_count  
id ty                                                                     
0  拖网    0.439489       0.0             15  228728.818841             23  
1  拖网    3.377653       0.0            106  245956.329870            138  

[2 rows x 26 columns]

In [3]:
names_x_x=['c_x','min_x','max_x','median_x','model_x','sub_x','c_y','min_y','max_y','median_y','model_y','sub_y','avg_v','stdev_v','median_v','entropy_v','model_v','d_v','avg_d','stdev_d','median_d','entropy_d','model_d','d_d','t','d_xy']
names_y='ty'
all_df.columns=names_x_x

all_df=all_df.reset_index()

In [4]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

d = defaultdict(LabelEncoder)
df = all_df.apply(lambda x:d[x.name].fit_transform(x) if type(x[0]) is str or math.isnan(x[0])  else x)

In [5]:
names_x=['c_x','min_x','max_x','median_x','model_x','sub_x','c_y','min_y','max_y','median_y','model_y','sub_y','avg_v','stdev_v','median_v','entropy_v','model_v','d_v','avg_d','stdev_d','median_d','entropy_d','model_d','d_d','t','d_xy']
x_train = df.loc[:,names_x].values
y_train = df.loc[:,names_y].values

from imblearn.over_sampling import RandomOverSampler
from collections import Counter
'''
ros = RandomOverSampler(random_state=0)
x_train, y_train = ros.fit_sample(x_train, y_train)
sorted(Counter(y_train).items())
'''

'\nros = RandomOverSampler(random_state=0)\nx_train, y_train = ros.fit_sample(x_train, y_train)\nsorted(Counter(y_train).items())\n'

In [38]:
df

,id,ty,c_x,min_x,max_x,median_x,model_x,sub_x,c_y,min_y,...,model_v,d_v,avg_d,stdev_d,median_d,entropy_d,model_d,d_d,t,d_xy
0,0,2,6.119351e+06,6.152038e+06,6.118352e+06,6.118352e+06,6.118352e+06,33686.667453,5.130494e+06,5.130781e+06,...,0.000,21.0,4.613527,21.247770,0.0,0.439489,0.000000,15,2.287288e+05,23
1,1,2,6.091460e+06,6.102450e+06,6.049472e+06,6.102450e+06,6.102450e+06,52978.013345,5.094050e+06,5.112874e+06,...,0.000,57.0,56.153247,91.449382,0.0,3.377653,0.000000,106,2.459563e+05,138
2,2,2,6.183011e+06,6.183191e+06,6.182482e+06,6.183090e+06,6.183090e+06,708.835147,5.193682e+06,5.193696e+06,...,0.215,23.0,123.356223,123.097127,71.0,5.577224,0.000000,95,4.064073e+05,4
3,3,2,5.239159e+06,5.287805e+06,5.228590e+06,5.229843e+06,5.229843e+06,59214.738740,4.601532e+06,4.608628e+06,...,0.000,60.0,121.134328,121.758165,91.0,5.080121,0.000000,116,2.826654e+05,100
4,4,1,7.062005e+06,7.070797e+06,7.049394e+06,7.061679e+06,7.060750e+06,21402.484584,6.116389e+06,6.136033e+06,...,0.320,57.0,139.067332,121.130025,112.0,6.076657,0.000000,154,2.361452e+05,162
5,5,2,6.488945e+06,6.503900e+06,6.388833e+06,6.496570e+06,6.496570e+06,115066.764852,5.576252e+06,5.603770e+06,...,0.000,50.0,130.461333,105.256336,127.0,6.036464,0.000000,147,1.686507e+05,93
6,6,2,6.058099e+06,6.079949e+06,6.032505e+06,6.058808e+06,6.069079e+06,47444.023130,5.028613e+06,5.047377e+06,...,3.890,46.0,142.972081,99.220841,104.5,7.437507,14.000000,206,5.766650e+06,392
7,7,1,6.759621e+06,6.986976e+06,6.587996e+06,6.811245e+06,6.589755e+06,398980.613694,5.805508e+06,5.814691e+06,...,0.000,87.0,168.327586,127.595550,247.5,5.718537,0.000000,140,1.813637e+05,198
8,8,2,6.201416e+06,6.215849e+06,6.181299e+06,6.203378e+06,6.199943e+06,34549.985805,5.139750e+06,5.166370e+06,...,3.805,58.0,138.114754,99.758217,165.5,5.936654,40.000000,118,6.207814e+06,333
9,9,2,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,0.000000,5.112531e+06,5.112531e+06,...,0.000,3.0,0.000000,0.000000,0.0,0.000000,0.000000,1,1.593053e+05,1


In [17]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

models = []
for index, (train_idx, test_idx) in enumerate(fold.split(x_train, y_train)):
    print(len(train_idx))
    train_set = lgb.Dataset(x_train[train_idx], y_train[train_idx])
    test_set = lgb.Dataset(x_train[test_idx], y_train[test_idx])
    params = {
    'n_estimators': 800,
    'boosting_type': 'rf', #'dart',#'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'num_leaves': 100,
    'max_depth': -1,
    'max_bin': 255,
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1
    }
    model = lgb.train(params, train_set, valid_sets=[train_set,test_set], verbose_eval=200)
    models.append(model)
    test_pred = model.predict(x_train[test_idx])
    test_pred_v = np.argmax(test_pred, axis=1)
    test_y =  y_train[test_idx]
    print(index, 'val f1', metrics.f1_score(test_y, test_pred_v, average='macro'))


5600


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	training's multi_logloss: 0.329281	valid_1's multi_logloss: 0.406343
[400]	training's multi_logloss: 0.32929	valid_1's multi_logloss: 0.406391
[600]	training's multi_logloss: 0.329151	valid_1's multi_logloss: 0.406046
[800]	training's multi_logloss: 0.329126	valid_1's multi_logloss: 0.406248
0 val f1 0.8200557336785533
5600
[200]	training's multi_logloss: 0.329598	valid_1's multi_logloss: 0.41734
[400]	training's multi_logloss: 0.328949	valid_1's multi_logloss: 0.416203
[600]	training's multi_logloss: 0.328974	valid_1's multi_logloss: 0.416204
[800]	training's multi_logloss: 0.329142	valid_1's multi_logloss: 0.41595
1 val f1 0.8178944116985866
5600
[200]	training's multi_logloss: 0.332797	valid_1's multi_logloss: 0.390311
[400]	training's multi_logloss: 0.332959	valid_1's multi_logloss: 0.389983
[600]	training's multi_logloss: 0.332651	valid_1's multi_logloss: 0.389867
[800]	training's multi_logloss: 0.332431	valid_1's multi_logloss: 0.389432
2 val f1 0.8469286170008564
5600
[200

In [46]:
all_test_df = pd.read_sql_query('''select id,concat(substr(t,1,9),'0:00') as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  , concat(max(x),'-',max(y))  as xy
from tianchi_01_test  group by id,concat(substr(t,1,9),'0:00')  ''',conn)

test_df=all_test_df.groupby(['id']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num],'y': [np.mean, np.max,np.min,np.median,model_num,sub_num],
                                     'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count] ,'d':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count],
                                       't':t_p,
                                        'xy':distinct_count
                                      })
test_df.columns=names_x_x
test_df=test_df.reset_index()

In [47]:
x_test = test_df.loc[:,names_x].values
test_y= search.predict(x_test)
test_df['p_ty']=test_y
test_df['p_ty_str']=d['ty'].inverse_transform(test_y)
test_y_p= search.predict_proba(x_test)
test_df['proba_0']=test_y_p[:,0]
test_df['proba_1']=test_y_p[:,1]
test_df['proba_2']=test_y_p[:,2]

In [48]:
result_df=test_df[['id','p_ty_str']]
result_df.to_csv("tianchi01_result_0203_GBDT.csv",index=False,header=0)


In [49]:
result_df_p=test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]
result_df_p['proba_max']=result_df_p[['proba_0','proba_1','proba_2']].max(axis=1)
result_df_p=result_df_p[['id','p_ty_str','proba_max']]
result_df_p.to_csv("tianchi01_result_p_0203_GBDT.csv",index=False,header=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
